In [1]:
import simpy
import numpy as np
import pandas as pd

class stocking_facility(object):
    def __init__(self, env, node_id, is_source, initial_inv, ROP, base_stock,
                 upstream, hist_demand, default_lead_time, lead_time_delay):
        self.env = env
        self.name = "node" + str(node_id)
        self.isSource = is_source
        self.on_hand_inventory = initial_inv
        self.inventory_position = initial_inv
        self.ROP = ROP
        self.baseStock = base_stock
        self.upstream = upstream
        self.histDemand = hist_demand
        self.defaultLeadTime = default_lead_time
        self.leadTimeDelay = lead_time_delay
        self.order_q = []
        self.totalDemand = 0.0
        self.totalShipped = 0.0
        self.serviceLevel = 0.0
        self.avgOnHand = 0.0
        self.onHandMon = []
        self.demandMon = []
        self.ordersPlaced = []
        self.shipmentsFulfilled = []
        self.shipmentsDict = {}
        self.remaining_order = 0
        self.shipment = 0
        self.shipment_serve_cust = 0
        self.demand = 0
        self.active_prepare_replenishment = 0
        self.order_check_inv = 0
        self.dd_hist = []

        self.env.process(self.check_inventory())
        self.env.process(self.prepare_replenishment())
        self.env.process(self.serve_customer())

    def check_inventory(self):
        while True:
            yield self.env.timeout(1.0)
            if self.inventory_position <= 1.05 * self.ROP:  # add 5% to avoid rounding issues
                order_qty = self.baseStock - self.on_hand_inventory
                order = {"orderQty": order_qty, "requester": self}
                self.upstream.order_q.append(order)
                self.inventory_position += order_qty
                self.order_check_inv = order_qty
                # self.ordersPlaced.append((self.env.now, order_qty))  # Track when the order is placed

    def prepare_replenishment(self):
        while True:
            if len(self.order_q) > 0:
                self.active_prepare_replenishment = 1
                order = self.order_q.pop(0)
                shipment = min(order["orderQty"], self.on_hand_inventory)
                self.shipment = shipment
                if not self.isSource:
                    self.inventory_position -= shipment
                    self.on_hand_inventory -= shipment
                remaining_order = order["orderQty"] - shipment
                self.remaining_order = remaining_order
                if remaining_order:
                    while not self.on_hand_inventory >= remaining_order:
                        yield self.env.timeout(1.0)
                    if not self.isSource:
                        self.inventory_position -= remaining_order
                        self.on_hand_inventory -= remaining_order
                # Schedule the shipment with the correct lead time
                self.env.process(self.ship(order["orderQty"], order["requester"]))
                # self.shipmentsFulfilled.append((self.env.now, order["orderQty"]))  # Track fulfillment time
                # self.shipmentsDict[self.env.now + self.defaultLeadTime] = order["orderQty"]
            else:
                yield self.env.timeout(1.0)

    def ship(self, qty, requester):
        lead_time = requester.defaultLeadTime  # Set the expected lead time
        yield self.env.timeout(lead_time)
        requester.on_hand_inventory += qty
        # requester.inventory_position += qty  # Update inventory position when the shipment arrives

    def serve_customer(self):
        while True:
            self.onHandMon.append(self.on_hand_inventory)
            yield self.env.timeout(1.0)
            demand = np.random.choice(self.histDemand, replace=True)
            self.demand = demand
            self.totalDemand += demand
            shipment = min(demand, self.on_hand_inventory)
            self.shipment_serve_cust = shipment
            self.totalShipped += shipment
            self.on_hand_inventory -= shipment
            self.inventory_position -= shipment
            self.demandMon.append(demand)

def simulate_network(seedinit, num_nodes, network, initial_inv, ROP, base_stock, demand, lead_time, lead_time_delay, time_units):
    env = simpy.Environment()
    np.random.seed(seedinit)
    nodes = []
    data_records = []  # List to collect data for Excel
   
    for i in range(num_nodes):
        if i == 0:
            s = stocking_facility(env, i, 1, initial_inv[i], ROP[i], base_stock[i], None, np.zeros(100), lead_time[i], lead_time_delay)
        else:
            for j in range(num_nodes):
                if network[j][i] == 1:
                    s = stocking_facility(env, i, 0, initial_inv[i], ROP[i], base_stock[i], nodes[j], demand[:, i - 1], lead_time[i], lead_time_delay)
                    break
        nodes.append(s)

    # Run the simulation and print values after each time unit
    for t in range(time_units):
        env.run(until=t+1)
        print(f"\nTime Unit {t+1}:")
        for node in nodes:
            print(
                  f"DEMAND: {node.demand}, "
                  f"Node {node.name} - On-hand Inventory: {node.on_hand_inventory}, "
                  f"Inventory Position: {node.inventory_position}, "
                  f"ROP: {node.ROP}, "
                  f"Base Stock: {node.baseStock}, "
                  f"order_check_inv: {node.order_check_inv},"
                  f"active_prepare_replenishment: {node.active_prepare_replenishment}, "
                  f"Shipment: {node.shipment}, "
                  f"Remaining Order: {node.remaining_order}, "
                  f"Shipment_serve_cust: {node.shipment_serve_cust}, "
                  f"Total Demand: {node.totalDemand}, "
                  f"Total Shipped: {node.totalShipped}",
                  f"Orders Placed: {node.ordersPlaced}")
                  # f"Shipments Fulfilled: {node.shipmentsFulfilled}"
   
    return nodes

# Simulation parameters
seedinit = 42
num_nodes = 3
network = [[0, 1, 1], [0, 0, 0], [0, 0, 0]]
ROP = [0, 400, 250]
base_stock = [10000, 600, 400]
initial_inv = [10000, 500, 300]
demand = np.array([[10, 20, 15, 25, 30], [5, 10, 7, 12, 8]]).T
lead_time = [0, 3, 3]
lead_time_delay = [0, 0, 0]
time_units = 30

print(f"Initial Inventory: {initial_inv}")
print(f"ROP: {ROP}")
print(f"Base Stock: {base_stock}")
print(f"Lead Time: {lead_time}")
nodes = simulate_network(seedinit, num_nodes, network, initial_inv, ROP, base_stock, demand, lead_time, lead_time_delay, time_units)

Initial Inventory: [10000, 500, 300]
ROP: [0, 400, 250]
Base Stock: [10000, 600, 400]
Lead Time: [0, 3, 3]

Time Unit 1:
DEMAND: 0, Node node0 - On-hand Inventory: 10000, Inventory Position: 10000, ROP: 0, Base Stock: 10000, order_check_inv: 0,active_prepare_replenishment: 0, Shipment: 0, Remaining Order: 0, Shipment_serve_cust: 0, Total Demand: 0.0, Total Shipped: 0.0 Orders Placed: []
DEMAND: 0, Node node1 - On-hand Inventory: 500, Inventory Position: 500, ROP: 400, Base Stock: 600, order_check_inv: 0,active_prepare_replenishment: 0, Shipment: 0, Remaining Order: 0, Shipment_serve_cust: 0, Total Demand: 0.0, Total Shipped: 0.0 Orders Placed: []
DEMAND: 0, Node node2 - On-hand Inventory: 300, Inventory Position: 300, ROP: 250, Base Stock: 400, order_check_inv: 0,active_prepare_replenishment: 0, Shipment: 0, Remaining Order: 0, Shipment_serve_cust: 0, Total Demand: 0.0, Total Shipped: 0.0 Orders Placed: []

Time Unit 2:
DEMAND: 0.0, Node node0 - On-hand Inventory: 10000.0, Inventory Po

In [2]:
# nodes[1].demandMon

In [3]:
# Define the supply chain network
num_nodes = 6
network = np.zeros((num_nodes, num_nodes))
network[0, 1] = 1
network[1, 2] = 1
network[1, 3] = 1
network[3, 4] = 1
network[3, 5] = 1


ROP = [0, 400, 250, 150, 200, 180]
base_stock = [10000, 600, 400, 900, 600, 450]
initial_inv = [10000, 500, 300, 810, 540, 360]
demand = np.array([
    [10, 20, 15, 25, 30],
    [5, 10, 7, 12, 8],
    [0, 0, 0, 0, 0],
    [6, 12, 8, 15, 10],
    [4, 9, 6, 10, 7],
]).T
lead_time = [0, 3, 3, 3, 3, 3]
lead_time_delay = [0, 0, 0, 0, 0, 0]
time_units = 365

print(f"Initial Inventory: {initial_inv}")
print(f"ROP: {ROP}")
print(f"Base Stock: {base_stock}")
print(f"Lead Time: {lead_time}")
nodes = simulate_network(seedinit, num_nodes, network, initial_inv, ROP, base_stock, demand, lead_time, lead_time_delay, time_units)

Initial Inventory: [10000, 500, 300, 810, 540, 360]
ROP: [0, 400, 250, 150, 200, 180]
Base Stock: [10000, 600, 400, 900, 600, 450]
Lead Time: [0, 3, 3, 3, 3, 3]

Time Unit 1:
DEMAND: 0, Node node0 - On-hand Inventory: 10000, Inventory Position: 10000, ROP: 0, Base Stock: 10000, order_check_inv: 0,active_prepare_replenishment: 0, Shipment: 0, Remaining Order: 0, Shipment_serve_cust: 0, Total Demand: 0.0, Total Shipped: 0.0 Orders Placed: []
DEMAND: 0, Node node1 - On-hand Inventory: 500, Inventory Position: 500, ROP: 400, Base Stock: 600, order_check_inv: 0,active_prepare_replenishment: 0, Shipment: 0, Remaining Order: 0, Shipment_serve_cust: 0, Total Demand: 0.0, Total Shipped: 0.0 Orders Placed: []
DEMAND: 0, Node node2 - On-hand Inventory: 300, Inventory Position: 300, ROP: 250, Base Stock: 400, order_check_inv: 0,active_prepare_replenishment: 0, Shipment: 0, Remaining Order: 0, Shipment_serve_cust: 0, Total Demand: 0.0, Total Shipped: 0.0 Orders Placed: []
DEMAND: 0, Node node3 - On

In [ ]:
# pd.DataFrame({"Node 0":nodes[0].demandMon,
#               "Node 1":nodes[1].demandMon,
#               "Node 2":nodes[2].demandMon,
#               "Node 3":nodes[3].demandMon,
#               "Node 4":nodes[4].demandMon,
#               "Node 5":nodes[5].demandMon}).to_csv("demand_by_numpy.csv")